## GR5245 Homework 2

(a) Download the TensorFlow flowers dataset from tensorflow_datasets. Take the first 80% as 
the training set and the remaining 20% as the validation set. 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


In [2]:
(x_train, x_valid), info = tfds.load('tf_flowers',
                                     split=['train[:80%]',
                                            'train[80%:]'],
                                     with_info=True,
                                     as_supervised=True)


(b) Write a preprocessing function to resize the images (both in training and validation sets) to have 
64x64 pixel values and rescale all pixel values to have values between 0 and 1. Build a data pipeline 
which will generate mini-batches of 32 samples randomly selected from a buffer of 1000 items. 

In [3]:
def preprocess(image, label):
    image = tf.image.resize(image, size=(64, 64))/255.0
    return image, label


In [4]:
train_ds = x_train.map(preprocess).shuffle(1000).batch(32)
valid_ds = x_valid.map(preprocess).shuffle(1000).batch(32)

(c) Create a Keras model as shown in the table below. Compile the model using SGD with a constant 
learning rate of 0.001, sparse categorical entropy as loss function and sparse categorical accuracy as 
the metric. Train the model for 20 epochs using the data pipeline prepared in (b). Keep the training 
history for later use.  

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),
    tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])


In [6]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [7]:
history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20


2022-10-25 12:31:40.797196: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


92/92 [==============================] - 10s 107ms/step - loss: 1.5467 - sparse_categorical_accuracy: 0.2943 - val_loss: 1.4773 - val_sparse_categorical_accuracy: 0.3774
Epoch 2/20
92/92 [==============================] - 9s 102ms/step - loss: 1.4412 - sparse_categorical_accuracy: 0.3777 - val_loss: 1.4204 - val_sparse_categorical_accuracy: 0.3801
Epoch 3/20
92/92 [==============================] - 9s 100ms/step - loss: 1.3662 - sparse_categorical_accuracy: 0.4309 - val_loss: 1.3355 - val_sparse_categorical_accuracy: 0.4210
Epoch 4/20
92/92 [==============================] - 9s 102ms/step - loss: 1.2928 - sparse_categorical_accuracy: 0.4666 - val_loss: 1.2448 - val_sparse_categorical_accuracy: 0.5054
Epoch 5/20
92/92 [==============================] - 10s 106ms/step - loss: 1.2238 - sparse_categorical_accuracy: 0.4959 - val_loss: 1.1964 - val_sparse_categorical_accuracy: 0.5272
Epoch 6/20
92/92 [==============================] - 10s 103ms/step - loss: 1.1777 - sparse_categorical_accura

(d) Create a new model by adding batch normalization layers as shown below. Compile and train the 
model using the same hyper parameters as in (c). Keep the training history for later use. 

In [8]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),

    tf.keras.layers.Conv2D(64, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(5, activation="softmax")
])


In [9]:
model2.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [10]:
history2 = model2.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20
92/92 [==============================] - 11s 118ms/step - loss: 1.2040 - sparse_categorical_accuracy: 0.5133 - val_loss: 2.0114 - val_sparse_categorical_accuracy: 0.2166
Epoch 2/20
92/92 [==============================] - 11s 118ms/step - loss: 0.7671 - sparse_categorical_accuracy: 0.7364 - val_loss: 1.9036 - val_sparse_categorical_accuracy: 0.2248
Epoch 3/20
92/92 [==============================] - 11s 118ms/step - loss: 0.5564 - sparse_categorical_accuracy: 0.8512 - val_loss: 1.6258 - val_sparse_categorical_accuracy: 0.2929
Epoch 4/20
92/92 [==============================] - 11s 117ms/step - loss: 0.4203 - sparse_categorical_accuracy: 0.9149 - val_loss: 1.4178 - val_sparse_categorical_accuracy: 0.3965
Epoch 5/20
92/92 [==============================] - 11s 116ms/step - loss: 0.3238 - sparse_categorical_accuracy: 0.9550 - val_loss: 1.2049 - val_sparse_categorical_accuracy: 0.4850
Epoch 6/20
92/92 [==============================] - 11s 116ms/step - loss: 0.2491 - sparse_cate

(e) Write a data augmentation function that either flips an image vertically or adjust the contrast factor 
to 1.5 (with equal probability of 50%). Use this function to create a new dataset from the original 
training set in (a). Merge the new dataset and the original training set to form an enlarged training 
dataset. 

In [11]:
def augmentation(image, label):
    if np.random.choice((0, 1)):
        image = tf.image.flip_left_right(image)
    else:
        image = tf.image.adjust_contrast(image, 1.5)
    return image, label

In [12]:
train_aug = x_train.map(augmentation)

In [13]:
train_large = x_train.concatenate(train_aug)

(f) Similar to (b), build a data pipeline for the enlarged training set created in (e).  

In [14]:
train_ds_aug = train_large.map(preprocess).shuffle(1000).batch(32)

(g) Create a new model with same architecture and parameters as the model in (d). Compile and train 
the model for 20 epochs using the data pipeline prepared in (f).

In [15]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),

    tf.keras.layers.Conv2D(64, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(5, activation="softmax")
])

In [16]:
model3.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [17]:
history3 = model3.fit(train_ds_aug,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20
184/184 [==============================] - 22s 116ms/step - loss: 1.0092 - sparse_categorical_accuracy: 0.6213 - val_loss: 1.8698 - val_sparse_categorical_accuracy: 0.2207
Epoch 2/20
184/184 [==============================] - 21s 116ms/step - loss: 0.5118 - sparse_categorical_accuracy: 0.8719 - val_loss: 1.5136 - val_sparse_categorical_accuracy: 0.3501
Epoch 3/20
184/184 [==============================] - 21s 113ms/step - loss: 0.3140 - sparse_categorical_accuracy: 0.9595 - val_loss: 1.0645 - val_sparse_categorical_accuracy: 0.5940
Epoch 4/20
184/184 [==============================] - 21s 114ms/step - loss: 0.2180 - sparse_categorical_accuracy: 0.9842 - val_loss: 1.0327 - val_sparse_categorical_accuracy: 0.6049
Epoch 5/20
184/184 [==============================] - 21s 115ms/step - loss: 0.1643 - sparse_categorical_accuracy: 0.9927 - val_loss: 1.0504 - val_sparse_categorical_accuracy: 0.5913
Epoch 6/20
184/184 [==============================] - 22s 120ms/step - loss: 0.1276 -